In [2]:
from tensorflow import keras 
import Rede_Neural as rna
from Rede_Neural import NeuralNetwork
from Rede_Neural import Layer

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import pyplot
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from utils import plot

%matplotlib inline


In [3]:
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import Dense # type: ignore
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from keras.layers import Input

#from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from scikeras.wrappers import KerasClassifier  
#from sklearn.wrappers import KerasClassifier

from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
import pickle
import joblib


In [4]:
def remove_highly_correlated_columns(data, threshold=0.9):
    """
    Remove columns from a NumPy array where the correlation coefficient
    is greater than a specified threshold.

    Parameters:
    - data: np.ndarray, the input array.
    - threshold: float, the correlation coefficient threshold.

    Returns:
    - np.ndarray, the array with highly correlated columns removed.
    """
    if not isinstance(data, np.ndarray):
        raise ValueError("Input data must be a NumPy array.")
    
    corr_matrix = np.corrcoef(data, rowvar=False)
    to_remove = set()
    
    for i in range(corr_matrix.shape[0]):
        for j in range(i+1, corr_matrix.shape[0]):
            if abs(corr_matrix[i, j]) > threshold:
                to_remove.add(j)

    columns_to_keep = [i for i in range(data.shape[1]) if i not in to_remove]
    filtered_data = data[:, columns_to_keep]

    return filtered_data


In [5]:
def dataprep_full(dataset, target, split = True):
    X_train, X_test, y_train, y_test = None, None, None, None
    
    pearson_threshold = 0.9
    # Create a pipeline for standardization and PCA
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=0.95))
    ])

    dataset = remove_highly_correlated_columns(dataset, pearson_threshold)
    print('features n-correlacionadas: ', dataset.shape)

    if split:
        seed = 7
        test_size = 0.30
        X_train, X_test, y_train, y_test = train_test_split(dataset, target, test_size=test_size, random_state=seed)
        
        # Transform the features
        X_train = pipeline.fit_transform(X_train)
        X_test = pipeline.fit_transform(X_test)
        #print(X_train.min(axis=0), X_train.max(axis=0))
    else:
        dataset  = pipeline.fit_transform(dataset)

    return dataset, X_train, X_test, y_train, y_test


In [6]:
def dataprep_standardScaler(dataset, target, split = True):
    X_train, X_test, y_train, y_test = None, None, None, None
    
    pearson_threshold = 0.9
    # Create a pipeline for standardization and PCA
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        #('pca', PCA(n_components=0.95))
    ])

    dataset = remove_highly_correlated_columns(dataset, pearson_threshold)
    print('features n-correlacionadas: ', dataset.shape)

    if split:
        seed = 7
        test_size = 0.30
        X_train, X_test, y_train, y_test = train_test_split(dataset, target, test_size=test_size, random_state=seed)
        
        # Transform the features
        X_train = pipeline.fit_transform(X_train)
        X_test = pipeline.fit_transform(X_test)
        #print(X_train.min(axis=0), X_train.max(axis=0))
    else:
        dataset  = pipeline.fit_transform(dataset)

    return dataset, X_train, X_test, y_train, y_test

In [7]:
def dataprep_pearson(dataset, target, split = True):
    X_train, X_test, y_train, y_test = None, None, None, None
    
    pearson_threshold = 0.9
    dataset = remove_highly_correlated_columns(dataset, pearson_threshold)
    print('features n-correlacionadas: ', dataset.shape)

    if split:
        seed = 7
        test_size = 0.30
        X_train, X_test, y_train, y_test = train_test_split(dataset, target, test_size=test_size, random_state=seed)
    
    return dataset, X_train, X_test, y_train, y_test

In [8]:
def dataprep_split(dataset, target, split = True):
    X_train, X_test, y_train, y_test = None, None, None, None

    if split:
        seed = 7
        test_size = 0.30
        X_train, X_test, y_train, y_test = train_test_split(dataset, target, test_size=test_size, random_state=seed)
    
    return dataset, X_train, X_test, y_train, y_test

In [9]:
# Carrega a base de dados a partir de seu caminho
data = np.load('data/dataset.npz')
print(data.files)

XX_train = data['X_train']
yy_train = data['y_train']
yy_train = yy_train.reshape(-1,1) #reorganiza o array em um array 1 x 1
    
X = XX_train
y = yy_train

unseen_data = data['X_test']
print('features: ', unseen_data.shape)

['X_train', 'y_train', 'X_test', 'y_test']
features:  (100855, 16)


In [10]:
X, X_train, X_test, y_train, y_test = dataprep_standardScaler(X, y, split = True)

features n-correlacionadas:  (47777, 15)


In [11]:
unseen_data, _ , _, _, _ = dataprep_standardScaler(dataset=unseen_data, target=None, split=False)
print('features: ', unseen_data.shape)
input_dim, output_dim = unseen_data.shape[1], 1
print(input_dim, output_dim)

features n-correlacionadas:  (100855, 15)
features:  (100855, 15)
15 1


In [12]:
def save_result(predictions, submission_file):
    y_test_probs = predictions.ravel()
    # Set the best threshold on validation data
    best_threshold = 0.5
    # Apply the best threshold to make binary predictions
    predicted_classes = (y_test_probs > best_threshold).astype(int)

    
    unique_values, indices, counts = np.unique(predicted_classes, return_index=True, return_counts=True)
    print("Unique values:", unique_values)
    print("Indices:", indices)
    print("Counts:", counts)
    print("Proportion:", counts[0]/counts[1])
    
    
    # Create a DataFrame with zipped data and column names
    #num_samples = unseen_data.shape[0]
    
    df = pd.DataFrame(list(zip(range(1, len(predicted_classes) + 1), predicted_classes)), columns=['ID', 'Prediction'])
    #df = pd.DataFrame({'ID': np.arange(1, num_samples + 1),'Prediction': predicted_classes})
    df.to_csv(submission_file, index=False)
    # Print the predictions
    #print("Predicted probabilities:\n", y_test_probs)
    #print("Predicted classes:\n", predicted_classes)

# First approach

In [13]:
# Load the model using the appropriate function
loaded_model =  keras.models.load_model('bestANN/model_1.keras')

test_loss, test_accuracy = loaded_model.evaluate(X_test, y_test)
print('For testing dataset. Loss: {:.4f} and Accuracy: {:.2f}%'.format(test_loss, 100*test_accuracy))

predictions = loaded_model.predict(unseen_data)
save_result(predictions=predictions, submission_file='submission_1.csv')

ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense" is incompatible with the layer: expected axis -1 of input shape to have value 16, but received input with shape (None, 15)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 15), dtype=float32)
  • training=False
  • mask=None

# Second approch

In [ ]:
loaded_model =  keras.models.load_model('bestANN/model_2.keras')

test_loss, test_accuracy = loaded_model.evaluate(X_test, y_test)
print('For testing dataset. Loss: {:.4f} and Accuracy: {:.2f}%'.format(test_loss, 100*test_accuracy))

predictions = loaded_model.predict(unseen_data)
save_result(predictions=predictions, submission_file='submission_2.csv')

448/448 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8181 - loss: 0.3935
For testing dataset. Loss: 0.3857 and Accuracy: 82.04%
3152/3152 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step
Unique values: [0 1]
Indices: [0 1]
Counts: [54937 45918]
Proportion: 1.1964153491005707


# Third approach

In [ ]:
loaded_model = NeuralNetwork.load('bestANN/model_3.pkl')

y_test_probs = loaded_model.predict(X_test)
print('For testing dataset. Accuracy: {:.2f}%'.format(100*accuracy_score(y_test, y_test_probs > 0.5)))

predictions = loaded_model.predict(unseen_data)
save_result(predictions=predictions, submission_file='submission_3.csv')

For testing dataset. Accuracy: 81.25%
Unique values: [0 1]
Indices: [0 1]
Counts: [54290 46565]
Proportion: 1.1658971330398369


# Fourth approch

In [ ]:
loaded_model = NeuralNetwork.load('bestANN/model_4.pkl')

y_test_probs = loaded_model.predict(X_test)
print('For testing dataset. Accuracy: {:.2f}%'.format(100*accuracy_score(y_test, y_test_probs > 0.5)))

predictions = loaded_model.predict(unseen_data)
save_result(predictions=predictions, submission_file='submission_4.csv')

For testing dataset. Accuracy: 82.07%
Unique values: [0 1]
Indices: [0 5]
Counts: [56862 43993]
Proportion: 1.292523810606233
